In [13]:
import numpy as np
import pandas as pd
import math

In [14]:
def monthly_return(return_yearly:float):
  """
  Calculate the monthly return of an expected yearly return.
  """
  return pow(1 + return_yearly, 1 / 12) - 1

In [15]:
def automatic_investment(return_monthly: float, investment_monthly: float, horizon: float, initial_balance: float = 0):
  """
  Calculate the balance after a certain period of time with a certain monthly investment and monthly return.
  suppose the investment is made at the start of each month.
  
  initial_balance: default 0.
  
  """
  month_num = horizon * 12
  balance = initial_balance + investment_monthly

  for i in range(month_num):
    balance = balance * (1 + return_monthly) + investment_monthly

  return balance

In [16]:
def back_to_present(target:str, target_value: float, return_monthly: float, horizon: float, investment_monthly: float = 0, initial_balance: float = 0):
  """
  Calculate the monthly investment value or expected monthly return.
  suppose the investment is made at the start of each month.
  
  """
  # 要求return_monthly or investment monthly
  month_num = horizon * 12
  
  if target == "num":
    monthly_num = (target_value - initial_balance * pow(1 + return_monthly, month_num)) * \
                  return_monthly / (pow(1 + return_monthly, month_num - 1) - 1)
    return monthly_num
    
  elif target == "rate":
    pass
  
  else:
    raise ValueError("target should be either 'num' or 'rate'")
    
  

In [17]:
yearly_return = 0.05
investment_horizon = 20
monthly_investment = 10
initial_balance = 0

In [18]:
print(f"经过20年的总余额是： {automatic_investment(monthly_return(yearly_return), monthly_investment, investment_horizon)} \n \
共盈利 {automatic_investment(monthly_return(yearly_return), monthly_investment, investment_horizon) - investment_horizon*12*monthly_investment}")

经过20年的总余额是： 4084.5778302295025 
 共盈利 1684.5778302295025


In [19]:
print(f"每月需要投入 {back_to_present('num', 4000, monthly_return(yearly_return), investment_horizon, monthly_investment, initial_balance)}")

每月需要投入 9.921571130768124
